# Doğrusal Regresyon Egzersizleri

50 adet Startup'ın araştırma ve geliştirmeye yönelik harcaması, yönetime yönelik harcaması, pazarlama harcaması, kazandıkları kar miktarı ve kuruldukları lokasyon bilgisi bulunmaktadır. Amaç kar miktarını tahmin etmektir. Bu bir sayısal tahmin problemidir ve bağımlı değişkenimiz "Profit".

Numpy, matplotlib.pyplot, pandas ve seaborn kütüphanelerini çekirdeğe dahil edelim.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Dizinde bulunan veri çerçevemizi startups değişkenine atayalım. startups değişkenini df değişkenine kopyalayarak kullanmaya başlayalım.

In [ ]:
df = pd.read_csv("../input/various-expenses-and-the-profits-of-50-startups/50_Startups.csv")

İlk 5 gözlemini yazdıralım.

In [ ]:
df.head()

Veri çerçevesinin bilgilerini görüntüleyelim.

In [ ]:
df.info()

Kaç gözlem ve öznitelikten oluştuğunu görüntüleyelim.

In [ ]:
df.shape

Eksik verileri kontrol edelim.

In [ ]:
df.isna().sum()

Korelasyon matrisi çizdirelim.

In [ ]:
df.corr()

Seaborn ile korelasyon matrisinin ısı haritasını çizdirelim.

In [ ]:
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values);

R&D Spend ve Profit arasındaki korelasyonu daha iyi görebilmek için scatterplot çizdirelim.

In [ ]:
sns.scatterplot(x="R&D Spend",y="Profit",data=df)

Sayısal değişkenlerin dağılımını görmek için df üzerinden histogram çizdirelim.

In [ ]:
plt.figure(figsize=(15,8))
plt.hist([df["Profit"],df["R&D Spend"],df["Administration"],df["Marketing Spend"]],
         color=['#4CE84C','#E84CE6','#E84C6D','#4C6DE8'],
         label=["Kâr","Ar-Ge Harc.","Yönetim Harc.","Pazarlama Harc."],
         histtype='bar',
         bins=10,
         range=[0,400000])
plt.legend()
plt.show()


In [ ]:
#Startupların çoğu 100000-150000 arasında kâr etmişler.
#Pazarlamaya yönelik harcamalar 0 ile 400000 arasında dağılmış belli bir yığılma olmamış.

Veri çerçevesinin temel istatistik değerlerini görüntüleyelim.

In [ ]:
df.describe().T

State'a ait benzersiz değerleri görüntüleyelim.

In [ ]:
df["State"].unique()

get_dummies yardımıyla State'a dair kategorik öznitelik çıkarımlarında bulunalım. Çünkü State'ların birbirine üstünlüğü yok, nominaller. Ordinal değil.

In [ ]:
df_State = pd.get_dummies(df["State"])
df_State.head(10)

In [ ]:
df = pd.concat([df,df_State],axis =1)

In [ ]:
df.head(10)

State özniteliğini silip dummy olarak yaratılan State'lardan da birisini hariç tutarak veri çerçevemizi güncelleyelim.

In [ ]:
df.drop(["State","California"], axis = 1, inplace = True)

In [ ]:
df.head(10)

Veri çerçevemizi bağımlı ve bağımsız değişkenler olmak üzere bölütleyelim.

In [ ]:
X = df.drop("Profit", axis = 1) #Bağımlı değişkenler
y = df["Profit"] #Bağımsız değişkenler

Bağımlı ve bağımsız değişkenleri kontrol edelim.

In [ ]:
X.head(10) #Bağımlı değişkenler

In [ ]:
y.head(10) #Bağımsız değişkenler

Bu bağımlı ve bağımsız değişkenlerden train ve test olmak üzere 4 parça oluşturalım. Bunu yapmak için train_test_split kullanalım.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 1) #%30 Test %70 Eğitim

4 parça değişkeni kontrol edelim.

In [ ]:
X_train.head(10)

In [ ]:
X_test.head(10)

In [ ]:
y_test.head(10)

In [ ]:
y_train.head(10)

LinearRegression'u çekirdeğe dahil edip modeli inşa edelim.

In [ ]:
from sklearn.linear_model import LinearRegression

linear_regresion = LinearRegression()

Modeli eğitmek için bağımlı bağımsız değişkenlerden oluşturulan eğitim verilerini verelim.

In [ ]:
model = linear_regresion.fit(X_train, y_train)

Modele daha önce görmediği bağımlı test değişkenini tahmin ettirelim. Bu tahmin değerlerimizi y_pred değişkenine atayalım.

In [ ]:
y_pred = model.predict(X_test)
y_pred

Tahminleri ve gerçek değerleri bir veri çerçevesinde toplayıp üzerinde göz gezdirelim.

In [ ]:
test = pd.DataFrame()
test["Gerçek Kâr"] = y_test
test["Tahmin Edilen Kâr"] = y_pred
test["Mutlak Fark"] = abs(y_pred-y_test)

In [ ]:
test

In [ ]:
#Modelimizin tahmini ile gerçek değerler arasındaki fark 265-14841 arasında değişiyor. bu sayılar modelimizin başarısını ölçmemiz için tek başına bir fikir vermiyor.

sklearn bünyesinde barınan metrics'i çekirdeğe dahil edelim ve MAE, MSE, RMSE değerlerini görüntüleyelim.

In [ ]:
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(y_train, model.predict(X_train))
MSE #Ortalama kare hata (gerçek değerler ile modelden çıkan değerler arasındaki farkın karesi)

In [ ]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_test,model.predict(X_test))
MAE  #Ortalama Mutlak Hata

In [ ]:
import math
RMSE = math.sqrt(MSE)
RMSE 

Modelin R Squared değerini eğitim verileri üzerinden yazdıralım.

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_train, model.predict(X_train))

In [ ]:
#Modelimizin başarısı arttıkça r^2 değeri de 1'e yaklaşır 0.94 modelimiz için gerçekten iyi bir değer ancak unutmamız gereken şey sadece bir tane eğitim kümesi üzerinde eğittik modelimizi.

In [ ]:
# Cross Validation ile farklı eğitim ve test kümeleri için modelimizin başarısına bakalım böylelikle modelin başarısını daha sağlıklı ölçebiliriz.

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
cross_val_score(model, X_train, y_train, cv = 4, scoring = "r2")

In [ ]:
cross_val_score(model, X_train, y_train, cv = 4, scoring = "r2").mean()

In [ ]:
#R^2 skorumuz ilk seferde 0.94 çıkmıştı 
#Dört farklı test ve eğitim kümesi seçip r^2 değerlerinin ortalması 0.90 çıktı.


Dileyenler statsmodel kullanarak hangi özniteliklerin model için %95 güvenilirlikle ne kadar anlamlı olup olmadığına da bakabilir. Modelde bazı feature selection işlemleri yaparak tekrardan eğitip yeni sonuçlar mukayese edilebilir.

In [ ]:
import statsmodels.api as sm
model2 = sm.OLS(y, X).fit()
model2.summary()

In [ ]:
X_train=X_train.iloc[:,0:3]
X_train.head()

In [ ]:
cross_val_score(model, X_train, y_train, cv = 4, scoring = "r2").mean()

In [ ]:
X_opt = X.iloc[:,0:3]
regressor_OLS = sm.OLS(y,X_opt).fit()
regressor_OLS.summary()